In [16]:
import pandas as pd
import numpy as np
import yfinance
import ta

# Buy Low With Volume Strategy
This strategy taking the newest low in n periods as a trigger with Stochastic RSI oversold, then using the volume as a filter.

### Part 1: Buying Requirements
  1. Stochastic RSI K and D line must be in the oversold area (> n%)
  2. Stochastic RSI K line is below D line
  3. The stock made an n period low during this period of Stochastic RSI oversold
  4. Then check if the volume is above the volume simple moving avreage within a n periods 
  5. If above then Buy at close or at n% above new low

### Part 2: Selling Requirements
  1. Set Trailling stop to n% of high prices
  2. Set sell limit to a static n% gain of buy price or trailling of high prices

## Getting Price Data
Getting the price data of symbol using the yfinance library.

In [34]:
# Loading the 10 years prices data from yfinance
# start: 2010-01-01 end: 2020-12-31
# Symbol: AAPL
symbol = 'TSLA'
ticker = yfinance.Ticker(symbol)
prices = ticker.history(start='2010-01-01', end='2020-12-31', period='1d') 

In [35]:
prices

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,0,0.0
2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,0,0.0
2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,0,0.0
2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,0,0.0
2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,0,0.0
...,...,...,...,...,...,...,...
2020-12-23,632.200012,651.500000,622.570007,645.979980,33173000,0,0.0
2020-12-24,642.989990,666.090027,641.000000,661.770020,22865600,0,0.0
2020-12-28,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0


## Technical Analysis
Using the ta library to do technical analysis.

In [36]:
# Create Stochastic RSI K
stoch_rsi = ta.momentum.StochRSIIndicator(prices.Close, window=20)
stoch_k = stoch_rsi.stochrsi_k()
stoch_d = stoch_rsi.stochrsi_d()
stoch_k

Date
2010-06-29         NaN
2010-06-30         NaN
2010-07-01         NaN
2010-07-02         NaN
2010-07-06         NaN
                ...   
2020-12-23    0.014007
2020-12-24    0.067020
2020-12-28    0.124796
2020-12-29    0.174499
2020-12-30    0.256703
Name: stochrsi_k, Length: 2646, dtype: float64

In [37]:
# Create Simple Moving Average for volumes
volume_sma = ta.trend.sma_indicator(prices.Volume, window=10)
volume_sma

Date
2010-06-29           NaN
2010-06-30           NaN
2010-07-01           NaN
2010-07-02           NaN
2010-07-06           NaN
                 ...    
2020-12-23    67409670.0
2020-12-24    62987910.0
2020-12-28    61568270.0
2020-12-29    58655290.0
2020-12-30    58432740.0
Name: sma_10, Length: 2646, dtype: float64

## Finding the Buy Signal

In [38]:
# Finding lows for 200 period, 100 period, 50 period and 20 period
period_range = [10]
low = prices.Low
# Simple function to find the lowest low in n periods
def get_lowest_low(s, length):
  return s.rolling(length).min()
# Getting a list of lows in different periods
lows = [get_lowest_low(low, _) for _ in period_range]
# Turn the list into a pandas DataFrame with concat
lows = pd.concat(lows, axis=1)
lows.columns = [str(_) + '_day_low' for _ in period_range]
# Check out lows
lows

,10_day_low
Date,
2010-06-29,NaN
2010-06-30,NaN
2010-07-01,NaN
2010-07-02,NaN
2010-07-06,NaN
...,...
2020-12-23,566.340027
2020-12-24,596.799988
2020-12-28,605.000000


In [39]:
# Concat Low price with Stochastic RSI K and different period lows
columns = ['Low', 'StochRSI_K', 'StochRSI_D'] + lows.columns.tolist()
ndf = pd.concat([prices.Low, stoch_k, stoch_d, lows], axis=1)
# Dropna, with inplace=True so do have to reassign to the same variable
ndf.dropna(inplace=True)
# check the ndf 
ndf

,Low,stochrsi_k,stochrsi_d,10_day_low
Date,,,,
2010-08-27,3.900000,0.723538,0.701562,3.652000
2010-08-30,3.922000,0.722929,0.707645,3.666000
2010-08-31,3.866000,0.748208,0.731559,3.666000
2010-09-01,3.920000,0.847676,0.772938,3.666000
2010-09-02,4.062000,0.930286,0.842057,3.702000
...,...,...,...,...
2020-12-23,622.570007,0.014007,0.190066,566.340027
2020-12-24,641.000000,0.067020,0.098985,596.799988
2020-12-28,660.799988,0.124796,0.068608,605.000000


This is the logic for getting the buy signal for back test.

In [40]:
# Applying the strategy:
# Check if Stochastic RSI K is lower than 20% (.2), 
# Stochastic RSI D is lower than 20% (.2) and 
# Stochastic RSI K less than Stochastic RSI D
below = .8
low_stoch_rsi = (ndf.stochrsi_k < below) &  (ndf.stochrsi_d < below) & (ndf.stochrsi_k < ndf.stochrsi_d)
# Check if the volume is above the volume sma
good_volume = prices.Volume > volume_sma
# Check if the price is one of the lows
# Using numpy's check to aviod pandas error
is_new_low = np.array(ndf.iloc[:, 2:]) >= np.array(ndf.loc[:, ['Low']])
# Check lowest is lower than more than n periods
is_new_low = is_new_low.sum(1) > 0
print('There are {} new period low in the dataframe of {} period.'.format(
  is_new_low.sum(), len(ndf)))

There are 358 new period low in the dataframe of 2604 period.


In [41]:
# Check for days of possible buys 
# If the Stochastic RSI K is lower then 20% and has an new low
has_hit = pd.concat([low_stoch_rsi, 
                     pd.Series(is_new_low, index=ndf.index)], 
                   axis=1).sum(1) > 1
signals = np.arange(len(has_hit))[has_hit]
print('There are {} signals.'.format(len(signals)))

There are 286 signals.


## Back Test

In [42]:
# Triming prices to the same length as ndf
index = ndf.index
nprices = prices.loc[index, :]
ngood_volume = good_volume.loc[index]

In [47]:
# Backtest loop
is_holding = False
buy_at = 0
stop_limit = 0
stop_limit_pct = 1 - .4
sell_limit = 0
sell_limit_pct = 1 + .5
current_idx = 0
wins = []
num_of_trades = 0
for s in signals:
  # Continue when current index is greater s
  # was still holding
  if current_idx > s:
    continue
  # When there is a signal buy at close
  # getting the next price data and setting the stop limit and sell limit
  data = nprices.iloc[s:, :5]
  for i, (_, row) in enumerate(data.iloc[1:].iterrows()):
    i = i + 1
    if not is_holding:
      if i > 5:
        break
      volume = ngood_volume.iloc[s + i]
      # Buy when good volume
      if volume:
        buy_at = row.Close
        current_high = row.High
        stop_limit = current_high * stop_limit_pct
        sell_limit = buy_at * sell_limit_pct
        is_holding = True
      continue
    # check sell conditions
    is_sell = False
    if row.Open < stop_limit:
      # if open prices is less than stop limit it's a sell
      wp = row.Open / buy_at - 1
      is_sell = True
    elif row.Low < stop_limit:
      # if low prices is less than stop limit, stop limit has been triggered
      wp = stop_limit / buy_at - 1
      is_sell = True
    elif row.High > sell_limit:
      # Sell when sell limit is reached
      wp = sell_limit / buy_at - 1
      is_sell = True
    elif i+1 == len(data):
      wp = row.Close / buy_at - 1
      is_sell = True
    
    if is_sell:
      # Selling logics
      wins.append(wp)
      current_idx = s + i
      num_of_trades += 1
      is_holding = False
      break

    # Update stop limit when there are new highs
    if row.High > current_high:
      stop_limit = row.High * stop_limit_pct
      sell_limit = row.High * sell_limit_pct
      current_high = row.High  

## Statistics

In [48]:
# Checking the strategy gain
gains = np.array(wins)
strategy_gain = np.prod(gains + 1) - 1
print(f'This Strategy has a gaining rate of : {strategy_gain*100:.2f}%')
print(f'Total trades of {num_of_trades}.') 
win_rate = len(gains[gains > 0]) / len(gains)
avg_win_pct = gains[gains > 0].mean() if win_rate != 0 else 0
biggest_win = gains[gains > 0].max() if win_rate != 0 else 0
avg_lost_pct = gains[gains < 0].mean() if win_rate != 1 else 0
biggest_lost = gains[gains < 0].min() if win_rate != 1 else 0

print(f'''Strategy has a win rate of {win_rate*100:.2f}% 
with an average win percentage of {avg_win_pct*100:.2f}% 
with a biggest win of {biggest_win*100:.2f}%
with an average lost percentage of {avg_lost_pct*100:.2f}% 
with a biggest lost of {biggest_lost*100:.2f}%''')

This Strategy has a gaining rate of : 11656.02%
Total trades of 7.
Strategy has a win rate of 85.71% 
with an average win percentage of 221.35% 
with a biggest win of 712.34%
with an average lost percentage of -33.31% 
with a biggest lost of -33.31%


In [49]:
# Seeing how much can be gain through moment
static_gain = nprices.Close[-1] / nprices.Close[0] - 1
print(f'The static gain of {symbol} is :  {static_gain*100:.2f}%') 

The static gain of TSLA is :  17534.01%


In [50]:
# Seeing how much can be gain when sell at the highest price
highest_gain = nprices.High.max() / nprices.Close[0] - 1
print(f'The maximum gain of {symbol} is :  {highest_gain*100:.2f}%') 

The maximum gain of TSLA is :  17580.20%


As you can see this strategy has an advantage to the base strategy, but we can even make it more profitable with some modify to certain parameters. 

The parameters are:
 - below: for setting or adjusting which is the oversold area.
 - stop_limit_pct: for setting the risk level for either a small or larger gain.
 - sell_limit_pct; for a set percentage gain